In [1]:
import openai
import os
import sys
import termcolor
import codecs
import json
import importlib
sys.path.append("../")
from magpie.prompt_planner.prompts import mp_prompt_thinker_coder_muk as mptc
from magpie.prompt_planner.prompts import dg_command_enumerator as dgce
from magpie.prompt_planner import conversation
from magpie.prompt_planner import confirmation_safe_executor
from magpie.prompt_planner import task_configs
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import rlds
API_KEY = os.environ['CORRELL_API_KEY']

2024-07-02 15:14:15.681301: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 15:14:15.681330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 15:14:15.682263: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 15:14:15.687681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 15:14:16.353465: W tensorflow/compiler/tf2

In [ ]:
pddf = pd.read_csv("robot_logs/df_2024_07_01_18_02_32_id-0.csv")
tfdd = tf.data.Dataset.load("robot_logs/episode_2024_07_01_18_02_32_id-0")

In [2]:
import tensorflow as tf
rl = tf.data.Dataset.load("robot_logs/episode_2024_07_01_18_02_32_id-0")
metadata = {
    'timestamp': tf.float32,
    'actual_qd': tf.float32,
    'aperture': tf.float32,
    'gripper_vel': tf.float32,
    'contact_force': tf.float32,
    'image': tf.uint8
}

2024-07-02 15:14:19.710236: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-07-02 15:14:19.710264: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: corvus
2024-07-02 15:14:19.710286: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: corvus
2024-07-02 15:14:19.710402: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 525.147.5
2024-07-02 15:14:19.710436: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 525.147.5
2024-07-02 15:14:19.710440: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 525.147.5


In [3]:
episode = rlds.build_episode(rl, metadata)

In [4]:
episode

{'steps': <_LoadDataset element_spec=({'aperture': TensorSpec(shape=(), dtype=tf.float32, name=None), 'gripper_vel': TensorSpec(shape=(), dtype=tf.float32, name=None), 'contact_force': TensorSpec(shape=(), dtype=tf.float32, name=None), 'image': TensorSpec(shape=(480, 640, 3), dtype=tf.uint8, name=None), 'actual_qd': TensorSpec(shape=(6,), dtype=tf.float32, name=None), 'timestamp': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.bool, name=None))>,
 'timestamp': tf.float32,
 'actual_qd': tf.float32,
 'aperture': tf.float32,
 'gripper_vel': tf.float32,
 'contact_force': tf.float32,
 'image': tf.uint8}

In [9]:
# convert rl, which is a tf.data.Dataset, to a tensorflow dataset, tfds, object
rltfds = tfds.as_numpy(rl)

In [12]:
for i in rltfds:
    print(i)

({'aperture': 51.68621, 'gripper_vel': 0.0, 'contact_force': 0.0, 'image': array([[[ 24,  45,  51],
        [ 23,  45,  48],
        [ 23,  45,  46],
        ...,
        [ 29,  47,  51],
        [ 28,  45,  50],
        [ 28,  45,  50]],

       [[ 24,  45,  51],
        [ 25,  47,  51],
        [ 25,  47,  48],
        ...,
        [ 26,  45,  49],
        [ 24,  43,  47],
        [ 24,  43,  47]],

       [[ 22,  44,  52],
        [ 23,  46,  51],
        [ 23,  47,  49],
        ...,
        [ 27,  46,  49],
        [ 27,  46,  50],
        [ 26,  45,  49]],

       ...,

       [[ 43,  45,  50],
        [ 43,  46,  50],
        [ 39,  46,  48],
        ...,
        [131, 138, 141],
        [133, 140, 142],
        [134, 140, 141]],

       [[ 42,  46,  48],
        [ 43,  47,  49],
        [ 41,  48,  49],
        ...,
        [128, 136, 140],
        [127, 135, 137],
        [129, 135, 137]],

       [[ 42,  48,  48],
        [ 41,  47,  47],
        [ 39,  47,  46],
        ...,

In [14]:
rltfds.element_spec

({'aperture': f32[],
  'gripper_vel': f32[],
  'contact_force': f32[],
  'image': ui8[480 640 3],
  'actual_qd': f32[6],
  'timestamp': f32[]},
 f32[],
 f32[],
 bool_[])

In [13]:
episode_tfds = rlds.build_episode(rltfds, metadata)

# pd df to tfds 

In [44]:
tfds = tf.data.Dataset.from_tensor_slices(df.to_dict(orient='list'))

In [45]:
tfds

<_TensorSliceDataset element_spec={'timestamp': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_0': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_1': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_2': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_3': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_4': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_5': TensorSpec(shape=(), dtype=tf.float32, name=None), 'aperture': TensorSpec(shape=(), dtype=tf.float32, name=None), 'gripper_vel': TensorSpec(shape=(), dtype=tf.float32, name=None), 'contact_force': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

# pd df direct to rlds

In [61]:
# !pip install rlds
import rlds

episode_steps = df.to_dict(orient='records')

# Convert list of dictionaries to RLDS episode
episode = {
    'steps':[{
        'observation': {
            'timestamp': step['timestamp'],
            'actual_qd': [
                step['actual_qd_0'],
                step['actual_qd_1'],
                step['actual_qd_2'],
                step['actual_qd_3'],
                step['actual_qd_4'],
                step['actual_qd_5']
            ],
            'aperture': step['aperture'],
            'gripper_vel': step['gripper_vel'],
            'contact_force': step['contact_force'],
            'image': np.zeros([64, 64, 3])
        },
        'action': 1,  # Placeholder, assuming no action data is available
        'reward': 0,  # Placeholder, assuming no reward data is available
        'is_terminal': False  # Placeholder, assuming steps are not terminal
    } for step in episode_steps]
}

# Print the RLDS episode
print(episode)


{'steps': [{'observation': {'timestamp': 1719530032.843785, 'actual_qd': [0.001078, -2.469845, 2.349459, -1.955513, -1.612564, -3.034139], 'aperture': 27.80900566920235, 'gripper_vel': 0.0, 'contact_force': 0.0, 'image': array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [

In [62]:
# Convert the episode to a list of tuples
steps = [
    (
        step['observation'],  # observation
        step['action'],       # action
        step['reward'],       # reward
        step['is_terminal']   # is_terminal
    )
    for step in episode['steps']
]

# Generator function
def generator():
    for step in steps:
        yield step

# Define the dataset types and shapes
output_types = (
    {
        'timestamp': tf.float32,
        'actual_qd': tf.float32,
        'aperture': tf.float32,
        'gripper_vel': tf.float32,
        'contact_force': tf.float32,
        'image': tf.uint8
    },
    tf.float32,  # action
    tf.float32,  # reward
    tf.bool      # is_terminal
)

output_shapes = (
    {
        'timestamp': tf.TensorShape([]),
        'actual_qd': tf.TensorShape([6]),
        'aperture': tf.TensorShape([]),
        'gripper_vel': tf.TensorShape([]),
        'contact_force': tf.TensorShape([]),
        'image': tf.TensorShape([64, 64, 3])
    },
    tf.TensorShape([]),  # action
    tf.TensorShape([]),  # reward
    tf.TensorShape([])   # is_terminal
)

# Create the tf.data.Dataset
dataset = tf.data.Dataset.from_generator(
    generator,
    output_types=output_types,
    output_shapes=output_shapes
)

# Print the dataset
for element in dataset:
    print(element)



({'timestamp': <tf.Tensor: shape=(), dtype=float32, numpy=1719530000.0>, 'actual_qd': <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([ 1.078000e-03, -2.469845e+00,  2.349459e+00, -1.955513e+00,
       -1.612564e+00, -3.034139e+00], dtype=float32)>, 'aperture': <tf.Tensor: shape=(), dtype=float32, numpy=27.809006>, 'gripper_vel': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'contact_force': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'image': <tf.Tensor: shape=(64, 64, 3), dtype=uint8, numpy=
array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],


In [8]:
import json
path = "robot_logs/grasp_logs.json"

# get content after last newline of stdout
with open(path, 'w') as f:
    json.dump(gl.tolist(), f)



In [9]:
import codecs
obj_text = codecs.open(path, 'r', encoding='utf-8').read()
b_new = json.loads(obj_text)
a_new = np.array(b_new)

In [2]:
task_configs.ALL_TASKS

{'magpie': TaskConfig(client=None, prompts={'thinker_coder': <class 'magpie.prompt_planner.prompts.mp_prompt_thinker_coder_muk.PromptThinkerCoder'>, 'thinker_coder_phys': <class 'magpie.prompt_planner.prompts.mp_prompt_tc_phys.PromptThinkerCoderPhys'>, 'thinker_coder_vision': <class 'magpie.prompt_planner.prompts.mp_prompt_tc_vision.PromptThinkerCoderVision'>, 'thinker_coder_vision_phys': <class 'magpie.prompt_planner.prompts.mp_prompt_tc_vision_phys.PromptThinkerCoderVisionPhys'>}),
 'system': TaskConfig(client=None, prompts={'system_executor': <class 'magpie.prompt_planner.prompts.dg_system_executor.PromptSystemExecutor'>}),
 'command_enumerator': TaskConfig(client=None, prompts={'command_enumerator': <class 'magpie.prompt_planner.prompts.dg_command_enumerator.PromptCommandEnumerator'>})}

In [3]:
tc = task_configs.ALL_TASKS['command_enumerator']
prompt = tc.prompts['command_enumerator']
MODEL = "gpt-3.5-turbo"
# MODEL = "gpt-4-turbo"
safe_executor = confirmation_safe_executor.ConfirmationSafeExecutor()
openai.api_key = API_KEY


In [8]:
# user_command = "gently pick the five largest and ripest tomatos and three small avocados"
# try:
#     prompt_model = prompt(
#         None, executor=safe_executor
#     )
#     conv = conversation.Conversation(prompt_model, MODEL)
#     # while True:
#       # user_command = input(termcolor.colored("User: ", "red", attrs=["bold"]))
#     try:
#       response = conv.send_command(user_command)
#       print(termcolor.colored("Magpie: ", "green", attrs=["bold"]) + response + "\n")
#     except Exception as e:  # pylint: disable=broad-exception-caught
#       print("Planning failed, try something else... " + str(e) + "\n")
#       # continue
    
#       # try:
#       #   prompt_model.code_executor(response)
#       # except Exception as e:  # pylint: disable=broad-exception-caught
#       #   print("Execution failed, try something else... " + str(e) + "\n")

# except Exception as e:
#     print(e)
#     prompt_model = None

LLM0 queried
[start of enumeration]
{'objects': ['tomato', 'avocado'],
'object_qualities': {'tomato': ['ripe', 'large'], 'avocado': ['small']},
'manipulation': 'pick',
'manipulation_qualities': ['gently']}
[end of enumeration]

processing response
processing command enumeration
LLM0 processed response: {'objects': ['tomato', 'avocado'],
'object_qualities': {'tomato': ['ripe', 'large'], 'avocado': ['small']},
'manipulation': 'pick',
'manipulation_qualities': ['gently']}
Magpie: {'objects': ['tomato', 'avocado'],
'object_qualities': {'tomato': ['ripe', 'large'], 'avocado': ['small']},
'manipulation': 'pick',
'manipulation_qualities': ['gently']}



In [11]:
import ast
ast.literal_eval(response)

{'objects': ['tomato', 'avocado'],
 'object_qualities': {'tomato': ['ripe', 'large'], 'avocado': ['small']},
 'manipulation': 'pick',
 'manipulation_qualities': ['gently']}

In [8]:
response = """
[start of enumeration]
1. Objects: ["avocados"]
2. Object Qualities: {"avocados": ["large"]}
3. Manipulation: "pick"
4. Manipulation Qualities: ["gently", "ignore irrelevant options"]
[end of enumeration]
"""

In [9]:
import re
import ast
import json
command_enumeration = (
    re.split(
        "end of enumeration",
        re.split("start of enumeration", response, flags=re.IGNORECASE)[
            1
        ],
        flags=re.IGNORECASE,
    )[0]
    .strip("[")
    .strip("]")
    .strip()
    .strip("```")
)


In [10]:
command_enumeration

'1. Objects: ["avocados"]\n2. Object Qualities: {"avocados": ["large"]}\n3. Manipulation: "pick"\n4. Manipulation Qualities: ["gently", "ignore irrelevant options"]'

In [11]:
# get object(s) as list from command_enumeration, try to use ast.literal_eval
objects = ast.literal_eval(command_enumeration.split("\n")[0].split(":")[1].strip())

In [28]:
oqual = command_enumeration.split("\n")[1]
start_index = oqual.find('{')
end_index = oqual.rfind('}')
oqual = ast.literal_eval(oqual[start_index:end_index+1])
oqual


In [32]:
# get manipulation type from command_enumeration as string
manipulation = ast.literal_eval(command_enumeration.split("\n")[2].split(":")[1].strip())
manipulation

'pick'

In [33]:
# get manipulation qualities from command_enumeration as list
mqual = ast.literal_eval(command_enumeration.split("\n")[3].split(":")[1].strip())
mqual

['gently', 'ignore irrelevant options']

In [13]:
object_qualities = ast.literal_eval(command_enumeration.split("\n")[1].split(":")[1].strip())

SyntaxError: '{' was never closed (<unknown>, line 1)

In [56]:
# get object(s) as list from command enumeration, after '1. Objects: ...' and before '2. Object Qualities: ...'
objects_search = re.search(r'1\. Objects:\s*\[(.*?)\]', command_enumeration)
if objects_search:
    objects = [obj.strip() for obj in objects_search.group(1).split(',')]
else:
    objects = []


In [57]:
objects

["'avocados'"]

In [58]:
qualities_match = re.search(r'2\. Object Qualities:\s*({.*?})', command_enumeration)
if qualities_match:
    object_qualities = json.loads(qualities_match.group(1).replace("'", '\"'))
else:
    object_qualities = {}

print("Object Qualities:", object_qualities)


Object Qualities: {'avocados': ['large']}


: 

In [32]:
tc = task_configs.ALL_TASKS['magpie']
tc.prompts['thinker_coder']


magpie.prompt_planner.prompts.mp_prompt_thinker_coder_muk.PromptThinkerCoder

In [6]:
import numpy as np
from PIL import Image
import base64
import io
import cv2 

def encode_image(img):
    # Convert NumPy array to base64 encoded string
    return base64.b64encode(img).decode('utf-8')


i = encode_image(np.array(Image.open("static/favicon.jpg")))

In [7]:
MODEL = "gpt-3.5-turbo"
import dataclasses
from typing import Any

In [8]:
@dataclasses.dataclass(frozen=True)
class TaskConfig:
  client: None
  prompts: dict[str, type[Any]]

ALL_TASKS = {
    'magpie': TaskConfig(
        client=None,
        prompts={
            'thinker_coder': mptc.PromptThinkerCoder,
        },
    ),
}


In [17]:
safe_executor = confirmation_safe_executor.ConfirmationSafeExecutor()
task_config = ALL_TASKS['magpie']
openai.api_key = API_KEY
prompt = task_config.prompts['thinker_coder']
MODEL = "gpt-3.5-turbo"

In [26]:
try:
    prompt_model = prompt(
        None, executor=safe_executor
    )
    conv = conversation.Conversation(prompt_model, MODEL)
    while True:
      user_command = input(termcolor.colored("User: ", "red", attrs=["bold"]))
      try:
        response = conv.send_command(user_command)
        print(termcolor.colored("Magpie: ", "green", attrs=["bold"]) + response + "\n")
      except Exception as e:  # pylint: disable=broad-exception-caught
        print("Planning failed, try something else... " + str(e) + "\n")
        continue
      
      try:
        prompt_model.code_executor(response)
      except Exception as e:  # pylint: disable=broad-exception-caught
        print("Execution failed, try something else... " + str(e) + "\n")

except Exception as e:
    print(e)
    prompt_model = None

LLM0 queried
[start of description]
* This is a new grasp.
* This grasp should be for picking up a grape.
* This is a complete grasp.
* This grasp is for an object with low compliance.
* The object has an approximate mass of 5 grams.
* The object has an approximate spring constant of 50 Newtons per meter.
* The gripper and object have an approximate friction coefficient of 0.2.
* This grasp should set the goal aperture to 5 mm.
* Based on object mass and friction coefficient, grasp should initially set the contact force to 1 Newton.
* If the gripper slips, this grasp should close an additional 1 mm.
* If the gripper slips, this grasp should increase the output force by 0.005 Newtons.
[end of description]

LLM1 queried
```python
from magpie.gripper import Gripper
G = Gripper()

import numpy as np

new_task = True

if new_task:
    G.reset_parameters()

goal_aperture = 5
complete_grasp = True
initial_force = (5 * 9.81) / 0.2
additional_closure = 1
spring_constant = 50
additional_force = 

KeyboardInterrupt: 